# Instrucciones adicionales en SQL
Existen instrucciones adicionales en SQL que permitirán no solo escribir un código más legible, sino uno que permita extraer información de una o más tablas, siempre y cuando estas esten relacionadas de alguna forma (sistema de información relacional)

## Creacion de una tabla en memoria RAM
Un posible recurso en SQLite es el uso de la memoria RAM para crear una tabla temporal. Esta tabla existirá en la memoria mientras el programa este en ejecución y no se halla cerrado la conexión.

En este ejemplo, la tabla Clientes tendrá un campos especificado como PRIMARY KEY. Al momento de insertar los datos, si no se especifica información para esta campo, este se rellenará con un valor incremental.

In [17]:
import sqlite3

# Datos a cargar en las tablas
data_clientes = [('Odontologia Pesquera', 'Dina Mita', 'Surco'),
                 ('Cebicheria Mar y Machas', 'Elvio Lado', 'La Victoria'),
                 ('La Pasta Basica', 'Susana Oria', 'Barranco'),
                 ('EC Sistemas', 'Enrique Cido', 'San Isidro'),
                 ('Veterinaria El Gato Felix', 'Estela Gartija', 'Barranco'),]

data_ordenes = [('101922', 2, '02-11-2018', 1200),
                ('101911', 4, '04-04-2019', 800),
                ('100111', 3, '11-05-2019', 540),
                ('101921', 2, '31-04-2019', 450),
                ('110192', 6, '06-07-2019', 810),
                ('110181', 1, '01-11-2019', 1110),
                ('111911', 3, '25-10-2019', 282),
                ('110019', 6, '22-07-2019', 740),
                ('112211', 5, '30-04-2019', 1200),
                ('102822', 3, '17-07-2019', 570),
                ('102823', 2, '22-12-2019', 980)]


Una vez creada la conexión con la base de datos en RAM y establecido el cursor, ejecutaremos instrucciones sobre el cursor (no utilizaremos un bloque with para evitar que se cierren ciertas instancias en la base de datos y asi mantener la tabla temporal en memoria). Se crearán tablas y se insertarán datos en estas para realizar las operaciones.

In [20]:
# Se crea la base de datos temporal en memoria
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

# Se crea la tabla Clientes
cur.execute("""CREATE TABLE IF NOT EXISTS Clientes (
                            clienteId INTEGER PRIMARY KEY, 
                            nombre TEXT, 
                            contacto TEXT, 
                            direccion TEXT)""")


# Se crea la tabla Ordenes
cur.execute("""CREATE TABLE IF NOT EXISTS Ordenes (
                            ordenId INTEGER PRIMARY KEY, 
                            clienteId INTEGER, 
                            fecha TEXT, 
                            monto REAL, 
                            FOREIGN KEY (clienteId) REFERENCES Clientes(clienteId))""")

# Se insertan valores en las tablas
cur.executemany("""INSERT INTO Clientes (
                            nombre, 
                            contacto, 
                            direccion) 
                   VALUES (?, ?, ?)""", data_clientes)

cur.executemany("""INSERT INTO Ordenes (
                            ordenId, 
                            clienteId, 
                            fecha, 
                            monto) 
                    VALUES (?, ?, ?, ?)""", data_ordenes)

# Se muestra el contenido de las tablas Clientes y Ordenes
print("----------------------------------------------------------------")
print("                       TABLAS DE DATOS")
print("----------------------------------------------------------------")
select_data_clientes = cur.execute("SELECT * FROM Clientes").fetchall()   # Concluye con la busqueda
select_data_ordenes = cur.execute("SELECT * FROM Ordenes").fetchall()    # Concluye con la busqueda

print("TABLA Clientes:")
print("==============")
for data_query in select_data_clientes:
    print(data_query)

print()

print("TABLA Ordenes:")
print("=============")
for data_query in select_data_ordenes:
    print(data_query)

print("----------------------------------------------------------------\n")

----------------------------------------------------------------
                       TABLAS DE DATOS
----------------------------------------------------------------
TABLA Clientes:
(1, 'Odontologia Pesquera', 'Dina Mita', 'Surco')
(2, 'Cebicheria Mar y Machas', 'Elvio Lado', 'La Victoria')
(3, 'La Pasta Basica', 'Susana Oria', 'Barranco')
(4, 'EC Sistemas', 'Enrique Cido', 'San Isidro')
(5, 'Veterinaria El Gato Felix', 'Estela Gartija', 'Barranco')

TABLA Ordenes:
(100111, 3, '11-05-2019', 540.0)
(101911, 4, '04-04-2019', 800.0)
(101921, 2, '31-04-2019', 450.0)
(101922, 2, '02-11-2018', 1200.0)
(102822, 3, '17-07-2019', 570.0)
(102823, 2, '22-12-2019', 980.0)
(110019, 6, '22-07-2019', 740.0)
(110181, 1, '01-11-2019', 1110.0)
(110192, 6, '06-07-2019', 810.0)
(111911, 3, '25-10-2019', 282.0)
(112211, 5, '30-04-2019', 1200.0)
----------------------------------------------------------------



## ORDER BY
Permite ordenar los resultados obtenidos por medio de un query en función de una columna. El orden puede ser ascendente (ASC, por defecto) o descencdente (DESC)

In [10]:
# Ejempo de ORDER BY
query = "SELECT * FROM Clientes ORDER BY nombre"   # ASC | DESC
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])   # Esta linea retorna los nombres de los campos
for data_query in query_res:
    print(data_query)


>> SELECT * FROM Clientes ORDER BY nombre
['CLIENTEID', 'NOMBRE', 'CONTACTO', 'DIRECCION']
(2, 'Cebicheria Mar y Machas', 'Elvio Lado', 'La Victoria')
(4, 'EC Sistemas', 'Enrique Cido', 'San Isidro')
(3, 'La Pasta Basica', 'Susana Oria', 'Barranco')
(1, 'Odontologia Pesquera', 'Dina Mita', 'Surco')
(5, 'Veterinaria El Gato Felix', 'Estela Gartija', 'Barranco')


## BETWEEN
Permite construir una instrucción más legible, reemplazando los multiples AND

In [9]:
# Ejemplo de BETWEEN (equivalente a multiples AND)
query = "SELECT * FROM Ordenes WHERE monto BETWEEN 500 AND 800 ORDER BY ordenId"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query)


>> SELECT * FROM Ordenes WHERE monto BETWEEN 500 AND 800 ORDER BY ordenId
['ORDENID', 'CLIENTEID', 'FECHA', 'MONTO']
('100111', 3, '11-05-2019', 540.0)
('101911', 4, '04-04-2019', 800.0)
('102822', 3, '17-07-2019', 570.0)
('110019', 6, '22-07-2019', 740.0)


## IN
Permite construir una instrucción más legible, reemplazando los multiples OR

In [11]:
# Ejempo con IN (equivalente a multiples OR)
query = "SELECT * FROM Clientes WHERE direccion IN ('Barranco', 'Surco') ORDER BY ClienteId"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query)


>> SELECT * FROM Clientes WHERE direccion IN ('Barranco', 'Surco') ORDER BY ClienteId
['CLIENTEID', 'NOMBRE', 'CONTACTO', 'DIRECCION']
(1, 'Odontologia Pesquera', 'Dina Mita', 'Surco')
(3, 'La Pasta Basica', 'Susana Oria', 'Barranco')
(5, 'Veterinaria El Gato Felix', 'Estela Gartija', 'Barranco')


## NOT
Todas las instrucciones SQL pueden contener la palabra reservada NOT para negar la lógica de la consulta

In [12]:
query = "SELECT * FROM Clientes WHERE direccion NOT IN ('Barranco', 'Surco') ORDER BY ClienteId"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query)


>> SELECT * FROM Clientes WHERE direccion NOT IN ('Barranco', 'Surco') ORDER BY ClienteId
['CLIENTEID', 'NOMBRE', 'CONTACTO', 'DIRECCION']
(2, 'Cebicheria Mar y Machas', 'Elvio Lado', 'La Victoria')
(4, 'EC Sistemas', 'Enrique Cido', 'San Isidro')


## JOIN
La instrucción JOIN permite obtener diferentes campos de diferentes tablas que estan relacionadas a traves de un campo (PRIMARY KEY contra FOREIGN KEY).

In [12]:
# Ejemplo de JOIN
query = "SELECT Ordenes.ordenId, Clientes.nombre, Ordenes.fecha \
FROM Ordenes JOIN Clientes \
ON Ordenes.clienteId = Clientes.clienteId ORDER BY Ordenes.ordenId"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query)


>> SELECT Ordenes.ordenId, Clientes.nombre, Ordenes.fecha FROM Ordenes JOIN Clientes ON Ordenes.clienteId = Clientes.clienteId ORDER BY Ordenes.ordenId
['ORDENID', 'NOMBRE', 'FECHA']
('100111', 'La Pasta Basica', '11-05-2019')
('101911', 'EC Sistemas', '04-04-2019')
('101921', 'Cebicheria Mar y Machas', '31-04-2019')
('101922', 'Cebicheria Mar y Machas', '02-11-2018')
('102822', 'La Pasta Basica', '17-07-2019')
('102822', 'Cebicheria Mar y Machas', '22-12-2019')
('110181', 'Odontologia Pesquera', '01-11-2019')
('111911', 'La Pasta Basica', '25-10-2019')
('112211', 'Veterinaria El Gato Felix', '30-04-2019')


Se muestran ejemplos que utilizan JOIN combinado con las demás instrucciones

In [13]:
# Que clientes tiene OC por montos entre 500 y 800 soles?
query = "SELECT Clientes.nombre, Ordenes.monto \
FROM Clientes JOIN Ordenes \
ON Clientes.clienteId = Ordenes.clienteId \
WHERE Ordenes.monto BETWEEN 500 AND 800"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query)


>> SELECT Clientes.nombre, Ordenes.monto FROM Clientes JOIN Ordenes ON Clientes.clienteId = Ordenes.clienteId WHERE Ordenes.monto BETWEEN 500 AND 800
['NOMBRE', 'MONTO']
('EC Sistemas', 800.0)
('La Pasta Basica', 540.0)
('La Pasta Basica', 570.0)


In [14]:
# Que clientes han puesto OC en Abril del 2019 por montos mayores a 500 soles?
query = "SELECT Ordenes.ordenId, Clientes.nombre, Ordenes.fecha, Ordenes.monto \
FROM Clientes JOIN Ordenes \
ON Clientes.clienteId = Ordenes.clienteId \
WHERE fecha LIKE '%-04-%' AND Ordenes.monto > 500"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query) 


>> SELECT Ordenes.ordenId, Clientes.nombre, Ordenes.fecha, Ordenes.monto FROM Clientes JOIN Ordenes ON Clientes.clienteId = Ordenes.clienteId WHERE fecha LIKE '%-04-%' AND Ordenes.monto > 500
['ORDENID', 'NOMBRE', 'FECHA', 'MONTO']
('101911', 'EC Sistemas', '04-04-2019', 800.0)
('112211', 'Veterinaria El Gato Felix', '30-04-2019', 1200.0)


In [15]:
# Cuales son las OC de los clientes ubicados en el distrito de Barranco? 
query = "SELECT Ordenes.ordenId, Clientes.nombre, Clientes.direccion \
FROM Clientes JOIN Ordenes \
ON Clientes.clienteId = Ordenes.clienteId \
WHERE Clientes.direccion = 'Barranco'"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query)


>> SELECT Ordenes.ordenId, Clientes.nombre, Clientes.direccion FROM Clientes JOIN Ordenes ON Clientes.clienteId = Ordenes.clienteId WHERE Clientes.direccion = 'Barranco'
['ORDENID', 'NOMBRE', 'DIRECCION']
('100111', 'La Pasta Basica', 'Barranco')
('111911', 'La Pasta Basica', 'Barranco')
('112211', 'Veterinaria El Gato Felix', 'Barranco')
('102822', 'La Pasta Basica', 'Barranco')


## SUM, AVG
Tanto SUM como AVG permite obtener resultados de una columna. Si la consulta retorna una lista de valores (como en el ejemplo, los montos de unas ordenes de compra), las operaciones anteriores retornarán la suma o el promedio de estos valores, respectivamente.

In [16]:
# Cuanto se facturo en el Mes de Abril de 2019? (SUM)
query = "SELECT SUM (Ordenes.monto) \
FROM Clientes JOIN Ordenes \
ON Clientes.clienteId = Ordenes.clienteId \
WHERE Ordenes.fecha LIKE '%04-2019%'"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query) 


>> SELECT SUM (Ordenes.monto) FROM Clientes JOIN Ordenes ON Clientes.clienteId = Ordenes.clienteId WHERE Ordenes.fecha LIKE '%04-2019%'
['SUM (ORDENES.MONTO)']
(2450.0,)


In [17]:
# Cual fue la facturacion promedio del año 2019? (AVG)
query = "SELECT AVG (Ordenes.monto) \
FROM Clientes JOIN Ordenes \
ON Clientes.clienteId = Ordenes.clienteId \
WHERE fecha LIKE '%-2019%'"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query)


>> SELECT AVG (Ordenes.monto) FROM Clientes JOIN Ordenes ON Clientes.clienteId = Ordenes.clienteId WHERE fecha LIKE '%-2019%'
['AVG (ORDENES.MONTO)']
(741.5,)


## ORDER BY, LIMIT
Cuando se retornan un conjunto de registros de una consulta, estos pueden agruparse alrededor de un campo, de forma tal que los demás campos se resumen de acuerdo a unas reglas internas de SQL. En el ejemplo, se otienen la lista de clientes y montos comprados. Al apgrupar con GROUP BY los resultados por los nombres de los clientes, los montos se suman. El resultado de una búsqueda también pueden limitarse con la instrucción LIMIT N.

In [18]:
# Cuales son nuestos clientes TOP 3
query = "SELECT Clientes.nombre, Ordenes.monto \
FROM Clientes JOIN Ordenes \
ON Clientes.clienteId = Ordenes.clienteId \
GROUP BY Clientes.nombre \
ORDER BY Ordenes.monto DESC \
LIMIT 3"
query_res = cur.execute(query)

print("\n>>", query)
print([description[0].upper() for description in query_res.description])
for data_query in query_res:
    print(data_query) 


>> SELECT Clientes.nombre, Ordenes.monto FROM Clientes JOIN Ordenes ON Clientes.clienteId = Ordenes.clienteId GROUP BY Clientes.nombre ORDER BY Ordenes.monto DESC LIMIT 3
['NOMBRE', 'MONTO']
('Veterinaria El Gato Felix', 1200.0)
('Odontologia Pesquera', 1110.0)
('Cebicheria Mar y Machas', 980.0)
